# Visualización de Datos con Python

**Curso:** CD2001B - Diagnóstico para Líneas de Acción  
**Semana 4:** Visualización de Datos y Dashboards  
**Objetivo:** Crear visualizaciones profesionales con matplotlib, seaborn y plotly para tu ONG

---

## 📋 Contenido

1. KPI Cards (Tarjetas de métricas clave)
2. Gráficos de tendencias temporales
3. Gráficos de distribución y comparación
4. Visualizaciones geográficas
5. Gráficos interactivos con Plotly
6. Dashboard completo en Python

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Configuración de estilo Tec de Monterrey
TEC_COLORS = {
    'azul_reflex': '#0062A4',
    'azul_oscuro': '#003E7E',
    'azul_claro': '#009FDA',
    'verde': '#8CC63F',
    'naranja': '#FF6F31',
    'gris_oscuro': '#58595B',
    'gris_medio': '#939598'
}

plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette([TEC_COLORS['azul_reflex'], TEC_COLORS['azul_claro'], 
                 TEC_COLORS['verde'], TEC_COLORS['naranja']])

%matplotlib inline

print("✅ Librerías importadas")

## Cargar Datos

In [ ]:
# Cargar datos preparados
# df = pd.read_csv('../datos/datos_looker_principal.csv')

# Datos de ejemplo
np.random.seed(42)
dates = pd.date_range(start='2024-01-01', end='2024-06-30', freq='D')
n = len(dates)

df = pd.DataFrame({
    'fecha': dates,
    'satisfaccion': np.random.normal(7.5, 1.5, n).clip(1, 10),
    'tiempo_atencion': np.random.normal(25, 10, n).clip(5, 60),
    'municipio': np.random.choice(['Puebla', 'Cholula', 'Atlixco', 'Tehuacán'], n),
    'genero': np.random.choice(['F', 'M'], n),
    'grupo_edad': np.random.choice(['Niños', 'Jóvenes', 'Adultos', 'Adultos Mayores'], n),
    'tipo_servicio': np.random.choice(['Servicio A', 'Servicio B', 'Servicio C'], n)
})

df['mes'] = df['fecha'].dt.month
df['mes_nombre'] = df['fecha'].dt.strftime('%B')
df['satisfecho'] = (df['satisfaccion'] >= 8).astype(int)

print(f"Datos cargados: {len(df)} registros")

## 1. KPI Cards (Tarjetas de Métricas Clave)

Visualización de métricas principales tipo dashboard.

In [ ]:
# Calcular KPIs principales
kpis = {
    'Total Atenciones': len(df),
    'Satisfacción Promedio': df['satisfaccion'].mean(),
    '% Satisfechos': (df['satisfecho'].sum() / len(df)) * 100,
    'Tiempo Promedio (min)': df['tiempo_atencion'].mean()
}

# Visualización estilo dashboard
fig, axes = plt.subplots(1, 4, figsize=(16, 4))
fig.suptitle('KPIs Principales - ONG', fontsize=18, fontweight='bold', y=1.05)

colors = [TEC_COLORS['azul_reflex'], TEC_COLORS['verde'], 
          TEC_COLORS['naranja'], TEC_COLORS['azul_claro']]

for idx, (key, value) in enumerate(kpis.items()):
    ax = axes[idx]
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.axis('off')
    
    # Fondo colorido
    rect = plt.Rectangle((0.05, 0.2), 0.9, 0.7, 
                          facecolor=colors[idx], alpha=0.2, edgecolor=colors[idx], linewidth=3)
    ax.add_patch(rect)
    
    # Título
    ax.text(0.5, 0.75, key, ha='center', va='center', 
            fontsize=12, fontweight='bold', color=colors[idx])
    
    # Valor
    if isinstance(value, float):
        display_value = f"{value:.1f}"
    else:
        display_value = f"{value:,}"
    
    ax.text(0.5, 0.45, display_value, ha='center', va='center',
            fontsize=28, fontweight='bold', color=colors[idx])

plt.tight_layout()
plt.show()

## 2. Gráficos de Tendencias Temporales

In [ ]:
# Tendencia mensual de satisfacción
tendencia_mensual = df.groupby('mes_nombre')['satisfaccion'].agg(['mean', 'std']).reset_index()
tendencia_mensual = tendencia_mensual.iloc[:6]  # Primeros 6 meses

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 5))

# Gráfico 1: Línea de tendencia
ax1.plot(tendencia_mensual['mes_nombre'], tendencia_mensual['mean'], 
         marker='o', linewidth=3, markersize=10, color=TEC_COLORS['azul_reflex'], label='Promedio')
ax1.fill_between(range(len(tendencia_mensual)), 
                 tendencia_mensual['mean'] - tendencia_mensual['std'],
                 tendencia_mensual['mean'] + tendencia_mensual['std'],
                 alpha=0.2, color=TEC_COLORS['azul_claro'], label='±1 Desv. Est.')
ax1.axhline(8, color=TEC_COLORS['verde'], linestyle='--', linewidth=2, label='Meta: 8.0')
ax1.set_title('Tendencia de Satisfacción Mensual', fontsize=14, fontweight='bold')
ax1.set_xlabel('Mes', fontsize=12)
ax1.set_ylabel('Satisfacción Promedio', fontsize=12)
ax1.legend()
ax1.grid(True, alpha=0.3)
ax1.tick_params(axis='x', rotation=45)

# Gráfico 2: Barras de atenciones por mes
atenciones_mes = df.groupby('mes_nombre').size().reset_index(name='count')
atenciones_mes = atenciones_mes.iloc[:6]
ax2.bar(atenciones_mes['mes_nombre'], atenciones_mes['count'], 
        color=TEC_COLORS['azul_claro'], edgecolor=TEC_COLORS['azul_oscuro'], linewidth=2)
ax2.set_title('Volumen de Atenciones Mensuales', fontsize=14, fontweight='bold')
ax2.set_xlabel('Mes', fontsize=12)
ax2.set_ylabel('Número de Atenciones', fontsize=12)
ax2.tick_params(axis='x', rotation=45)
ax2.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## 3. Gráficos de Distribución y Comparación

In [ ]:
# Satisfacción por grupo de edad
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Boxplot
order = ['Niños', 'Jóvenes', 'Adultos', 'Adultos Mayores']
sns.boxplot(data=df, x='grupo_edad', y='satisfaccion', order=order, ax=ax1,
            palette=[TEC_COLORS['azul_reflex'], TEC_COLORS['azul_claro'], 
                    TEC_COLORS['verde'], TEC_COLORS['naranja']])
ax1.axhline(8, color='red', linestyle='--', linewidth=2, alpha=0.7, label='Meta: 8.0')
ax1.set_title('Distribución de Satisfacción por Grupo de Edad', fontsize=14, fontweight='bold')
ax1.set_xlabel('Grupo de Edad', fontsize=12)
ax1.set_ylabel('Satisfacción', fontsize=12)
ax1.legend()
ax1.tick_params(axis='x', rotation=45)

# Barras agrupadas
satisfaccion_grupo = df.groupby('grupo_edad')['satisfaccion'].mean().reindex(order)
bars = ax2.barh(order, satisfaccion_grupo, 
                color=[TEC_COLORS['azul_reflex'], TEC_COLORS['azul_claro'], 
                      TEC_COLORS['verde'], TEC_COLORS['naranja']])
ax2.axvline(df['satisfaccion'].mean(), color='red', linestyle='--', linewidth=2, 
            label=f'Promedio General: {df["satisfaccion"].mean():.2f}')
ax2.set_title('Satisfacción Promedio por Grupo', fontsize=14, fontweight='bold')
ax2.set_xlabel('Satisfacción Promedio', fontsize=12)
ax2.set_ylabel('Grupo de Edad', fontsize=12)
ax2.legend()
ax2.grid(True, alpha=0.3, axis='x')

# Agregar valores en las barras
for i, bar in enumerate(bars):
    width = bar.get_width()
    ax2.text(width + 0.1, bar.get_y() + bar.get_height()/2, 
             f'{width:.2f}', ha='left', va='center', fontweight='bold')

plt.tight_layout()
plt.show()

## 4. Visualizaciones Geográficas

In [ ]:
# Mapa de calor por municipio
municipio_stats = df.groupby('municipio').agg({
    'satisfaccion': 'mean',
    'tiempo_atencion': 'mean',
    'fecha': 'count'
}).rename(columns={'fecha': 'total_atenciones'})

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Gráfico 1: Satisfacción por municipio
municipio_stats['satisfaccion'].sort_values().plot(kind='barh', ax=axes[0],
                                                     color=TEC_COLORS['azul_reflex'])
axes[0].set_title('Satisfacción Promedio por Municipio', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Satisfacción', fontsize=12)
axes[0].grid(True, alpha=0.3, axis='x')

# Gráfico 2: Volumen por municipio
municipio_stats['total_atenciones'].sort_values().plot(kind='barh', ax=axes[1],
                                                         color=TEC_COLORS['verde'])
axes[1].set_title('Total de Atenciones por Municipio', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Atenciones', fontsize=12)
axes[1].grid(True, alpha=0.3, axis='x')

# Gráfico 3: Tiempo de atención
municipio_stats['tiempo_atencion'].sort_values().plot(kind='barh', ax=axes[2],
                                                        color=TEC_COLORS['naranja'])
axes[2].set_title('Tiempo Promedio de Atención por Municipio', fontsize=14, fontweight='bold')
axes[2].set_xlabel('Minutos', fontsize=12)
axes[2].grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

## 5. Gráficos Interactivos con Plotly

Plotly permite crear gráficos interactivos ideales para presentaciones.

In [ ]:
# Gráfico de línea interactivo
tendencia_diaria = df.groupby('fecha')['satisfaccion'].mean().reset_index()

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=tendencia_diaria['fecha'],
    y=tendencia_diaria['satisfaccion'],
    mode='lines+markers',
    name='Satisfacción',
    line=dict(color=TEC_COLORS['azul_reflex'], width=2),
    marker=dict(size=4),
    hovertemplate='<b>Fecha:</b> %{x}<br><b>Satisfacción:</b> %{y:.2f}<extra></extra>'
))

# Línea de meta
fig.add_hline(y=8, line_dash="dash", line_color=TEC_COLORS['verde'],
              annotation_text="Meta: 8.0", annotation_position="right")

fig.update_layout(
    title='Tendencia Diaria de Satisfacción (Interactivo)',
    xaxis_title='Fecha',
    yaxis_title='Satisfacción',
    hovermode='x unified',
    template='plotly_white',
    height=500
)

fig.show()

In [ ]:
# Sunburst chart (gráfico de sol) para jerarquías
# Municipio → Grupo Edad → Tipo Servicio
sunburst_data = df.groupby(['municipio', 'grupo_edad', 'tipo_servicio']).size().reset_index(name='count')

fig = px.sunburst(
    sunburst_data,
    path=['municipio', 'grupo_edad', 'tipo_servicio'],
    values='count',
    color='count',
    color_continuous_scale='Blues',
    title='Distribución Jerárquica de Atenciones'
)

fig.update_layout(height=600)
fig.show()

## 6. Dashboard Completo en una Figura

Combina múltiples visualizaciones en un dashboard tipo infografía.

In [ ]:
# Dashboard completo con subplots
fig = plt.figure(figsize=(18, 12))
gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)

# Título general
fig.suptitle('Dashboard de KPIs - ONG', fontsize=20, fontweight='bold', y=0.98)

# 1. KPI Cards (fila superior)
kpi_axes = [fig.add_subplot(gs[0, i]) for i in range(3)]
kpi_values = [
    (f"{len(df):,}", "Total Atenciones", TEC_COLORS['azul_reflex']),
    (f"{df['satisfaccion'].mean():.2f}", "Satisfacción Promedio", TEC_COLORS['verde']),
    (f"{(df['satisfecho'].sum()/len(df)*100):.1f}%", "% Satisfechos", TEC_COLORS['naranja'])
]

for ax, (value, label, color) in zip(kpi_axes, kpi_values):
    ax.text(0.5, 0.6, value, ha='center', va='center', 
            fontsize=32, fontweight='bold', color=color)
    ax.text(0.5, 0.3, label, ha='center', va='center',
            fontsize=14, color=color)
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.axis('off')
    rect = plt.Rectangle((0.05, 0.1), 0.9, 0.8, facecolor=color, alpha=0.15,
                          edgecolor=color, linewidth=2)
    ax.add_patch(rect)

# 2. Tendencia temporal
ax1 = fig.add_subplot(gs[1, :])
tendencia_mensual = df.groupby('mes_nombre')['satisfaccion'].mean().iloc[:6]
ax1.plot(tendencia_mensual.index, tendencia_mensual.values, 
         marker='o', linewidth=3, markersize=10, color=TEC_COLORS['azul_reflex'])
ax1.axhline(8, color=TEC_COLORS['verde'], linestyle='--', linewidth=2, label='Meta')
ax1.set_title('Tendencia de Satisfacción Mensual', fontsize=14, fontweight='bold')
ax1.set_ylabel('Satisfacción')
ax1.legend()
ax1.grid(True, alpha=0.3)
ax1.tick_params(axis='x', rotation=45)

# 3. Por municipio
ax2 = fig.add_subplot(gs[2, 0])
municipio_stats['satisfaccion'].sort_values().plot(kind='barh', ax=ax2,
                                                     color=TEC_COLORS['azul_claro'])
ax2.set_title('Satisfacción por Municipio', fontsize=12, fontweight='bold')
ax2.set_xlabel('Satisfacción')
ax2.grid(True, alpha=0.3, axis='x')

# 4. Por grupo de edad
ax3 = fig.add_subplot(gs[2, 1])
grupo_stats = df.groupby('grupo_edad')['satisfaccion'].mean().sort_values()
grupo_stats.plot(kind='bar', ax=ax3, color=TEC_COLORS['verde'])
ax3.set_title('Satisfacción por Grupo de Edad', fontsize=12, fontweight='bold')
ax3.set_ylabel('Satisfacción')
ax3.tick_params(axis='x', rotation=45)
ax3.grid(True, alpha=0.3, axis='y')

# 5. Distribución de tiempo
ax4 = fig.add_subplot(gs[2, 2])
ax4.hist(df['tiempo_atencion'], bins=20, color=TEC_COLORS['naranja'], alpha=0.7, edgecolor='black')
ax4.axvline(df['tiempo_atencion'].mean(), color='red', linestyle='--', linewidth=2,
            label=f'Media: {df["tiempo_atencion"].mean():.1f} min')
ax4.set_title('Distribución de Tiempo de Atención', fontsize=12, fontweight='bold')
ax4.set_xlabel('Minutos')
ax4.set_ylabel('Frecuencia')
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.savefig('dashboard_completo.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Dashboard guardado como 'dashboard_completo.png'")

## 7. Exportar Gráficos para Presentación

In [ ]:
# Función para guardar gráficos de alta calidad
def guardar_grafico(fig, nombre, dpi=300):
    fig.savefig(f'{nombre}.png', dpi=dpi, bbox_inches='tight', facecolor='white')
    print(f"✅ Guardado: {nombre}.png")

# Crear gráfico limpio para presentación
fig, ax = plt.subplots(figsize=(12, 6))

tendencia_mensual = df.groupby('mes_nombre')['satisfaccion'].mean().iloc[:6]
ax.plot(tendencia_mensual.index, tendencia_mensual.values,
        marker='o', linewidth=4, markersize=12, color=TEC_COLORS['azul_reflex'],
        label='Satisfacción Promedio')
ax.axhline(8, color=TEC_COLORS['verde'], linestyle='--', linewidth=3, label='Meta: 8.0')
ax.fill_between(range(len(tendencia_mensual)), 8, tendencia_mensual.values,
                where=(tendencia_mensual.values >= 8), alpha=0.3, color=TEC_COLORS['verde'],
                label='Por encima de meta')

ax.set_title('Evolución de Satisfacción del Beneficiario', fontsize=18, fontweight='bold', pad=20)
ax.set_xlabel('Mes', fontsize=14, fontweight='bold')
ax.set_ylabel('Satisfacción (1-10)', fontsize=14, fontweight='bold')
ax.legend(fontsize=12, loc='best')
ax.grid(True, alpha=0.3, linestyle='--')
ax.tick_params(axis='both', labelsize=12)
ax.tick_params(axis='x', rotation=45)

plt.tight_layout()
guardar_grafico(fig, 'grafico_satisfaccion_presentacion')
plt.show()

## ✅ Checklist de Visualizaciones

Asegúrate de incluir en tu dashboard de Looker Studio:

**Elementos esenciales:**
- [ ] KPI cards con métricas principales
- [ ] Gráfico de tendencia temporal (línea)
- [ ] Comparación por categorías (barras)
- [ ] Distribución de variables clave (histograma/boxplot)
- [ ] Segmentación geográfica si aplica
- [ ] Filtros interactivos

**Mejores prácticas:**
- [ ] Usar colores consistentes (paleta Tec)
- [ ] Incluir líneas de meta/referencia
- [ ] Agregar etiquetas claras
- [ ] Mostrar valores en puntos clave
- [ ] No sobrecargar con demasiados gráficos

---

## 📚 Recursos Adicionales

- **Guía de tipos de gráficos:** `/Semana4/guias/guia_tipos_graficos.md`
- **Checklist de visualización:** `/Semana4/plantillas/checklist_visualizacion.md`
- **Ejemplo de dashboard:** `/Semana4/ejemplos/dashboard_ejemplo_ong.pdf`

---

**💡 TIP:** Usa estas visualizaciones como inspiración para tu dashboard en Looker Studio. Puedes recrear los mismos gráficos de forma interactiva.
